<a href="https://colab.research.google.com/github/ruan-narici/brazilian-ecommerce-analytics/blob/main/brazilian_ecommerce_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio de Projeto: Análise de Vendas e Entregas no E-commerce Brasileiro

Imports

In [4]:
import pandas as pd

Load datasets

In [5]:
df_customers = pd.read_csv("/content/drive/MyDrive/Documentos/Dataset/Brazilian E-Commerce Public Dataset by Olist/olist_customers_dataset.csv") #OK
df_geolocation = pd.read_csv("/content/drive/MyDrive/Documentos/Dataset/Brazilian E-Commerce Public Dataset by Olist/olist_geolocation_dataset.csv") #EXCLUDE
df_order_items = pd.read_csv("/content/drive/MyDrive/Documentos/Dataset/Brazilian E-Commerce Public Dataset by Olist/olist_order_items_dataset.csv") #OK
df_order_payments = pd.read_csv("/content/drive/MyDrive/Documentos/Dataset/Brazilian E-Commerce Public Dataset by Olist/olist_order_payments_dataset.csv") #OK
df_order_reviews = pd.read_csv("/content/drive/MyDrive/Documentos/Dataset/Brazilian E-Commerce Public Dataset by Olist/olist_order_reviews_dataset.csv") #OK
df_orders = pd.read_csv("/content/drive/MyDrive/Documentos/Dataset/Brazilian E-Commerce Public Dataset by Olist/olist_orders_dataset.csv") #OK
df_products = pd.read_csv("/content/drive/MyDrive/Documentos/Dataset/Brazilian E-Commerce Public Dataset by Olist/olist_products_dataset.csv") #OK
df_sellers = pd.read_csv("/content/drive/MyDrive/Documentos/Dataset/Brazilian E-Commerce Public Dataset by Olist/olist_sellers_dataset.csv") #OK
df_product_category_name_translation = pd.read_csv("/content/drive/MyDrive/Documentos/Dataset/Brazilian E-Commerce Public Dataset by Olist/product_category_name_translation.csv") #OK

Define Types

In [16]:
# Order Items
df_order_items["shipping_limit_date"] = pd.to_datetime(df_order_items["shipping_limit_date"])

# Order Reviews
df_order_reviews["review_creation_date"] = pd.to_datetime(df_order_reviews["review_creation_date"])
df_order_reviews["review_answer_timestamp"] = pd.to_datetime(df_order_reviews["review_answer_timestamp"])

# Orders
df_orders["order_purchase_timestamp"] = pd.to_datetime(df_orders["order_purchase_timestamp"])
df_orders["order_approved_at"] = pd.to_datetime(df_orders["order_approved_at"])
df_orders["order_delivered_carrier_date"] = pd.to_datetime(df_orders["order_delivered_carrier_date"])
df_orders["order_delivered_customer_date"] = pd.to_datetime(df_orders["order_delivered_customer_date"])
df_orders["order_estimated_delivery_date"] = pd.to_datetime(df_orders["order_estimated_delivery_date"])

Merge datasets

In [17]:
df_merged = df_orders.merge(df_order_items, on="order_id")
df_merged = df_merged.merge(df_customers, on="customer_id")
df_merged = df_merged.merge(df_sellers, on="seller_id")
df_merged = df_merged.merge(df_products, on="product_id")
df_merged = df_merged.merge(df_order_payments, on="order_id")
df_merged = df_merged.merge(df_product_category_name_translation, on="product_category_name")
df_merged = df_merged.merge(df_order_reviews, on="order_id")
df_merged.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'seller_zip_code_prefix', 'seller_city', 'seller_state',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'product_category_name_english', 'review_id',
       'review_score', 'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp'],
      dtype='object')

Selecting some columns

In [25]:
 df_final = df_merged[
    [
        "order_id",
        "customer_id",
        "order_status",
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "shipping_limit_date",
        "order_estimated_delivery_date",
        "product_id",
        "seller_id",
        "customer_city",
        "seller_city",
        "customer_state",
        "seller_state",
        "product_category_name_english",
        "payment_type",
        "price",
        "freight_value",
        "payment_value",
        "review_score",
        "review_creation_date",
        "review_answer_timestamp",
        ]
    ]

df_final

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,shipping_limit_date,order_estimated_delivery_date,product_id,...,seller_state,product_category_name_english,payment_type,price,freight_value,payment_value,review_score,review_creation_date,review_answer_timestamp,product_description_lenght
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-06 11:07:15,2017-10-18,87285b34884572647811a353c7ac498a,...,SP,housewares,credit_card,29.99,8.72,18.12,4,2017-10-11,2017-10-12 03:43:48,268.0
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-06 11:07:15,2017-10-18,87285b34884572647811a353c7ac498a,...,SP,housewares,voucher,29.99,8.72,2.00,4,2017-10-11,2017-10-12 03:43:48,268.0
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-06 11:07:15,2017-10-18,87285b34884572647811a353c7ac498a,...,SP,housewares,voucher,29.99,8.72,18.59,4,2017-10-11,2017-10-12 03:43:48,268.0
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-07-30 03:24:27,2018-08-13,595fac2a385ac33a80bd5114aec74eb8,...,SP,perfumery,boleto,118.70,22.76,141.46,4,2018-08-08,2018-08-08 18:37:50,178.0
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-08-13 08:55:23,2018-09-04,aa4383b373c6aca5d8797843e5594415,...,SP,auto,credit_card,159.90,19.22,179.12,5,2018-08-18,2018-08-22 19:07:58,232.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115604,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-02-12 13:10:37,2018-03-02,f1d4ce8c6dd66c47bbaa8c6781c2a923,...,SP,baby,credit_card,174.90,20.10,195.00,4,2018-03-01,2018-03-02 17:50:01,828.0
115605,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-05 15:04:16,2017-09-27,b80910977a37536adeddd63663f916ad,...,SP,home_appliances_2,credit_card,205.99,65.02,271.01,5,2017-09-22,2017-09-22 23:10:57,500.0
115606,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-01-12 21:36:21,2018-02-15,d1c427060a0f73f6b889a5c7c61f2ac4,...,MG,computers_accessories,credit_card,179.99,40.59,441.16,2,2018-01-26,2018-01-27 09:16:56,1893.0
115607,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-01-12 21:36:21,2018-02-15,d1c427060a0f73f6b889a5c7c61f2ac4,...,MG,computers_accessories,credit_card,179.99,40.59,441.16,2,2018-01-26,2018-01-27 09:16:56,1893.0


In [40]:
df_final.groupby("seller_state")["freight_value"].sum().sort_values(ascending=False).head(5)

,freight_value
seller_state,
SP,1524304.73
MG,218315.96
PR,204346.18
SC,110494.42
RJ,95826.02


In [41]:
df_final.groupby("seller_state")["seller_state"].count().sort_values(ascending=False).head(5)


,seller_state
seller_state,
SP,82417
MG,9014
PR,8964
RJ,4906
SC,4221


In [43]:
df_final.groupby("seller_state")["payment_value"].sum().sort_values(ascending=False).head(5)

,payment_value
seller_state,
SP,13102725.34
PR,1826605.49
MG,1524405.49
RJ,1075181.72
SC,880990.16
